컨볼루션 네트워크의 안전연결층에서는 3차원의 출력값을 1차원 벡터로 평탄화 시켜주는 작업을 수행하여 일반 신경망 연결처럼 출력층의 모든 노드와 연결시켜주는 역할을 수행한다

컨볼루션 함수
conv = tf.nn.conv2d(input, filter, strides, padding,...)

input : 컨볼루션 연산을 위한 입력 데이터이며 [batch, in_height, in_width, in_channels] 예를들어, 100 개의 배치로 묶은 28X28 크기의 흑백 이미지를 입력 으로 넣을경우 input 은 [100, 28,28, 1] 로 나타냄

filter : 컨볼루션 연산에 적용할 필터이며 [filter_height, filter_width, in_channels, out_channels] 예를들어, 필터 크기 3X3이며 입력채널 개수는 1이고 적용되는 필터 개수가 총 32개이면 filter 는 [3, 3, 1, 32] 로 나타냄

strides: 컨볼루션 연산을 위해 필터를 이동시키는 간격을 나타냄. 예를들어 [1, 1, 1, 1] 로 strides를 나타낸다면 컨볼루션 적용을 위해 1 칸씩 이동 필터를 이동하는것을 의미함

padding: ‘SAME’ 또는 ‘VALID’ 값을 가짐. padding=‘VALID’ 라면 컨볼루션 연산 공식에 의해서 가로/세로(차원) 크기가 축소된 결과가 리턴됨. 그러나 padding=‘SAME’ 으로 지정하면 입력 값의 가로/세로(차원) 크기와 같은 출력이 리턴되도록 작아진 차원 부분에 0 값을 채운 제로패딩을 수행함

* 입력채널 : 데이터의 이동 통로

pooling 함수 tf.nn.max_pool(value, ksize, strides, padding,...)

value: [batch, height, width, channels] 형식의 입력데이터. 일반적으로 relu 를 통과한 출력결과를 말한다

ksize: 컨볼루션 신경망에서 일반적인 ksize는 다음과 같이 [1, height, width, 1] 형태로 표시함. 예를 들어 ksize = [1, 2, 2, 1]이라면 2칸씩 이동하면서 출력결과 1 개를 만들어 낸다는 것을 의미함. 즉 4개 (2X2) 데이터 중에서 가장 큰 값 1 개를 찾아서 반환하는 역할을 수행함. 
만약 ksize = [1, 3, 3, 1] 이라고 하면 3칸씩 이동, 즉 9개 (3X3) 데이터 중에서 가장 큰 값을 찾는다는 의미임

strides: max pooling을 위해 윈도우를 이동시키는 간격을 나타냄. 예를들어 [1, 2, 2, 1] 로 strides를 나타낸다면 max pooling 적용을 위해 2 칸씩 이동하는 것을 의미함

padding: max pooling 에서의 padding 값은 max pooling 을 수행하기에는 데이터가 부족한 경우에 주변을 0 등으로 채워주는 역할을 함. 예를들어 max pooling 에서 풀링층으로 들어오는 입력데이터가 7X7 이고, 데이터를 2개씩 묶어 최대값을 찾아내는 연산을 하기에는 입력으로 주어진 데이터가 부족한 상황임 (즉, 최소 8X8 이어야 가능). 이때padding=‘SAME’ 이면, 부족한 데이터 부분을 0 등으로 채운 후에 데이터를 2개씩 묶어 최대값을 뽑아낼 수 있음

채널마다 바이어스가 존재해야함
필터가 32개면 바이어스도 32개


In [10]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

print("")
print("train.num = ", mnist.train.num_examples,
     ", test.num = ", mnist.test.num_examples,
     " , validation.num = ", mnist.validation.num_examples)
tf.__version__

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

train.num =  55000 , test.num =  10000  , validation.num =  5000


'1.15.0'

In [11]:
#Hyper Parameter

learning_rate = 0.0001
epochs = 30
batch_size = 100

#PlaceHolder
X = tf.placeholder(tf.float32, [None, 784])

T = tf.placeholder(tf.float32, [None, 10])

A1 = X_img = tf.reshape(X, [-1, 28, 28, 1])

In [12]:
#1번째 컨볼루션 층 3X3X32 필터
W2 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
b2 = tf.Variable(tf.random_normal([32]))

#1번째 컨볼루션 연산
C2 = tf.nn.conv2d(A1, W2, strides=[1, 1, 1, 1], padding = 'SAME')

#relu
Z2 = tf.nn.relu(C2 + b2)

# 1번째 max pooling을 통해 28 X 28 X 32 => 14 X 14 X 32
A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')

In [13]:
A2_flat = P2_flat = tf.reshape(A2, [-1, 14*14*32])

In [14]:
W3 = tf.Variable(tf.random_normal([14*14*32, 10], stddev = 0.01))
b3 = tf.Variable(tf.random_normal([10]))

Z3 = logits = tf.matmul(A2_flat, W3) + b3

y = A3 = tf.nn.softmax(Z3)

In [15]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = Z3, labels = T))

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [16]:
predicted_val = tf.equal(tf.argmax(A3, 1), tf.argmax(T, 1))

#batch_size X 10의 True, Flase를 1또는 0으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

In [17]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    start_time = datetime.now()
    
    for i in range(epochs):
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
            
            loss_val, _ = sess.run([loss, train], feed_dict = {X : batch_x_data, T: batch_t_data})
            
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)
                
    end_time = datetime.now()
    
    print("\nelapsed time = ", end_time = start_time)
    
    # Accuracy
    test_x_data = mnist.test.images
    test_t_data = mnist.test.labels
    
    accuract_val = sess.tun(accuracy, feed_dict = {X: test_x_data, T: test_t_data})
    
    print("\nAccuracy = ", accuracy_val)

epochs =  0 , step =  0 , loss_val =  3.0205503
epochs =  0 , step =  100 , loss_val =  2.3118703
epochs =  0 , step =  200 , loss_val =  2.3037217
epochs =  0 , step =  300 , loss_val =  2.3105526
epochs =  0 , step =  400 , loss_val =  2.2989044
epochs =  0 , step =  500 , loss_val =  2.3071725
epochs =  1 , step =  0 , loss_val =  2.3076527
epochs =  1 , step =  100 , loss_val =  2.308975
epochs =  1 , step =  200 , loss_val =  2.3028376
epochs =  1 , step =  300 , loss_val =  2.2948828
epochs =  1 , step =  400 , loss_val =  2.3090494
epochs =  1 , step =  500 , loss_val =  2.3038225
epochs =  2 , step =  0 , loss_val =  2.2996845
epochs =  2 , step =  100 , loss_val =  2.3048458
epochs =  2 , step =  200 , loss_val =  2.3072047
epochs =  2 , step =  300 , loss_val =  2.291049
epochs =  2 , step =  400 , loss_val =  2.3030562
epochs =  2 , step =  500 , loss_val =  2.2987397
epochs =  3 , step =  0 , loss_val =  2.314963
epochs =  3 , step =  100 , loss_val =  2.2916152
epochs =  3

KeyboardInterrupt: 